In [1]:
import psycopg2
import pandas as pd
import numpy as np

/Users/Jeremy/anaconda3/lib/python3.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [14]:
#conntect to aws database
connection = psycopg2.connect(
    host = 'changingroom-database1.cfiednof9fbl.us-east-2.rds.amazonaws.com',
    port = 5432,
    user = 'change_admin',
    password = 'ChangeFashion2020!',
    database='marketplace'
    )
cursor=connection.cursor()

In [13]:
sql = """
SELECT *
FROM clothes LIMIT 5
"""
pd.read_sql(sql, con=connection)['image_link_color']

"{'Multicolored': 'https://static.zara.net/photos///2021/I/0/1/p/7918/493/330/2/w/1920/7918493330_1_1_1.jpg?ts=1622792745210'}"

### Clothes Data

In [196]:
#Note: You always need to commit after executing a sql command
cursor.execute("""DROP TABLE clothes CASCADE""")

connection.commit()

In [197]:
#here is an example to create table materials with columns material_id of type text, chemistry of type float etc.
#each row of a table should ideally be uniquely identifiable by an id so always make sure to have an id column
cursor.execute("""CREATE TABLE clothes(
clothing_id text PRIMARY KEY,
display_name text,
color text ARRAY[20],
size text ARRAY[20],
price text,
product_url text,
image_link_color JSONB,
brand_name text,
description text,
scrapped_date date,
low_level text,
material_dict JSONB,
gender text,
OverallScore float);""")

connection.commit()

In [202]:
#option 1 - when the csv is in the exact form/schema of the table to upload
csv_file_name = 'clothing_df.csv'
sql = "COPY clothes FROM STDIN DELIMITER ',' CSV HEADER"
cursor.copy_expert(sql, open(csv_file_name, "r"))

connection.commit()

In [201]:
connection.rollback()

### ITEM HAS MATERIAL

In [4]:
sql = """
SELECT *
FROM item_has_material LIMIT 5
"""
pd.read_sql(sql, con=connection)

,id,clothing_id,material_id,part,percentage
0,186422792,20210630hm0,tex_cott_6,Shell,96%
1,735107864,20210630hm0,tex_elas_10,Shell,4%
2,121845571,20210630hm0,tex_cott_6,Shell,96%
3,695983975,20210630hm0,tex_elas_10,Shell,4%
4,749547351,20210630hm0,tex_cott_6,Shell,100%


In [207]:
item_df = pd.read_csv('item_has_material.csv')

In [210]:
np.random.seed(100)
item_df['id'] = np.random.randint(0,1000000000,size=(len(item_df)))
#item_df[item_df['id'].duplicated()]

In [211]:
item_df = item_df[['id','clothing_id', 'material_id', 'part', 'percentage']]

In [212]:
item_df.to_csv('item_has_material.csv',index=None)

In [5]:
#Note: You always need to commit after executing a sql command
cursor.execute("""DROP TABLE item_has_material CASCADE""")

connection.commit()

In [6]:
#here is an example to create table materials with columns material_id of type text, chemistry of type float etc.
#each row of a table should ideally be uniquely identifiable by an id so always make sure to have an id column
cursor.execute("""CREATE TABLE item_has_material(
id SERIAL PRIMARY KEY,
clothing_id text REFERENCES clothes(clothing_id),
material_id text REFERENCES materials(material_id),
part text,
percentage text);""")

connection.commit()

In [7]:
#option 1 - when the csv is in the exact form/schema of the table to upload
csv_file_name = 'item_has_material.csv'
sql = "COPY item_has_material FROM STDIN DELIMITER ',' CSV HEADER"
cursor.copy_expert(sql, open(csv_file_name, "r"))

connection.commit()

### PART COMPONENT VALUE

In [23]:
sql = """
SELECT *
FROM part_component_value LIMIT 5
"""
pd.read_sql(sql, con=connection)

,high_level,low_level,part,component_significance,component_value_percentage
0,shirts,T-Shirts,Padding,Tertiary,0%
1,shirts,T-Shirts,Cup Lining,Tertiary,0%
2,shirts,T-Shirts,Shell,Main,100%
3,shirts,T-Shirts,Net,Tertiary,5%
4,shirts,T-Shirts,Upper,Secondary,50%


In [15]:
#Note: You always need to commit after executing a sql command
cursor.execute("""DROP TABLE part_component_value CASCADE""")

connection.commit()

In [16]:
#here is an example to create table materials with columns material_id of type text, chemistry of type float etc.
#each row of a table should ideally be uniquely identifiable by an id so always make sure to have an id column
cursor.execute("""CREATE TABLE part_component_value(
PRIMARY KEY(high_level, low_level, part),
high_level text,
low_level text,
part text,
component_significance text,
component_value_percentage text);""")

connection.commit()

In [22]:
#option 1 - when the csv is in the exact form/schema of the table to upload
csv_file_name = 'part_component_value.csv'
sql = "COPY part_component_value FROM STDIN DELIMITER ',' CSV HEADER"
cursor.copy_expert(sql, open(csv_file_name, "r"))

connection.commit()

In [21]:
connection.rollback()

In [ ]:
['high_level', 'low_level', 'part', 'component_significance',
       'component_value_percentage']